In [1]:
import numpy as np
import scipy.stats as stats
import random

import math
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import ScalarFormatter

In [2]:
# Define Tableau 10 Colors
tableau_colors = [
    (31, 119, 180),  # Blue 0
    (255, 127, 14),  # Orange 1
    (44, 160, 44),   # Green 2
    (214, 39, 40),   # Red 3
    (148, 103, 189), # Purple 4 
    (140, 86, 75),   # Brown 5
    (227, 119, 194), # Pink 6
    (127, 127, 127), # Gray 7
    (188, 189, 34),  # Yellow 8
    (23, 190, 207),  # Cyan 9
]

# Normalize RGB values to range [0, 1]
tableau_colors = [(r / 255, g / 255, b / 255) for r, g, b in tableau_colors]

font = {'family': 'Georgia', 'color':  'black', 'weight': 'normal', 'size': 20}
title_font = {'family': 'Georgia', 'color':  'black', 'weight': 'bold', 'style': 'italic', 'size': 20}
suptitle_font = FontProperties(family='Georgia', weight='bold', size=22)
legend_font = FontProperties(family='Georgia', weight='normal', size=16)
tick_font = {'family': 'Georgia', 'size': 18}

      SUBROUTINE ELOSS(ENE,ANGL,ENEAFT)
      COMMON IX,ANUMBE
C *** CALCULATE ENERGY LOSS IN COLLISION, USE ISOTROPIC (S-WAVE) 
C *** APPROXIMATION. ROUND UP ANY ENERGY BELOW 0.1 EV TO 0.02 EV (THERMAL)
      IF(ENE.LT.1.E-7)THEN
      ENEAFT=2.E-8
      ELSE
C *** THIS IS THE MAXIMUM RECOIL ENERGY THAT CAN BE IMPARTED
      EREMAX=((4.*ANUMBE)/((1.+ANUMBE)**2.))*ENE
      CALL RANDM(RN1)
C *** THE PROBABILITY DISTRIBUTION FOR RECOIL ENERGIES IS FLAT
C *** FROM ZERO UP TO THE MAXIMUM 
      EREMAX=EREMAX*RN1
      ENEAFT=ENE-EREMAX
      ENDIF

      IF(ENE.LT.1.E-7)THEN
      CALL RANDM(RN1)
      ANGL=ACOS((2.*RN1)-1.)
      ELSE
      ETA=ACOS(SQRT((EREMAX/ENE)*(((1.+ANUMBE)**2.)/(4.*ANUMBE))))
      ANGL=ATAN((SIN(2.*ETA))/((1./ANUMBE)-COS(2.*ETA)))
      ENDIF
      
      RETURN
      END

In [16]:
def get_fission_neutron_energy():
    """USE VON NEUMANN'S METHOD ("HIT AND MISS") TO CALCULATE THE 
    ENERGY OF THE NEUTRON EMITED FROM A FISSION SOURCE."""
    alpha = 1.4 # MeV, fitting parameter
    while True:
        rand_energy = random.random() * 9.999 # MeV, 9.999 is higher than max possible
        prob = math.sqrt(rand_energy) * math.exp(-rand_energy / alpha)
        rand_prob = random.random() * 0.5 # probability, 0.5 is higher than max possible
        if rand_prob <= prob:
            fission_energy = rand_energy
            return fission_energy
    

In [1]:
def get_cross_sections(energy):
    """RETURNS THE APPROXIMATE CROSS SECTIONS (BARNS) FOR ABSORPTION AND 
    ELASTIC SCATTERING IN CARBON. HERE WE NEED THE ENERGY IN EV, NOT MEV."""
    e_MeV = energy * 1e6
    if e_MeV < 1e4:
        cross_sec_elastic = 5
    else:
        cross_sec_elastic = 10.5 - (1.346 * math.log10(e_MeV))
    cross_sec_elastic /= 1e24 
    
    if e_MeV < 1e3:
        cross_sec_absorb = (6.442e-4) * (e_MeV ** (-0.49421)) 
    elif e_MeV < 1e5:
        cross_sec_absorb = 1.5e-5
    elif e_MeV < 5e6:
        cross_sec_absorb = 2e-5
    else:
        cross_sec_absorb = (4e-6) * math.exp(e_MeV * 3.2189e-7) 
    cross_sec_absorb /= 1e24 
    
    return cross_sec_elastic, cross_sec_absorb

     SUBROUTINE EULER(EX,EY,EZ,ANGL,SX,SY,SZ)
C     THIS SUBROUTINE TAKES THE ORIGINAL LINEAR TRAJECTORY,
C     ROTATES IT TO LIE ALONG THE Z-AXIS, GENERATES A VECTOR
C     AT ZENITH ANGLE THETA = SCATTERING ANGLE 
C     AND AZIMUTHAL ANGLE FI = RANDOM * 2PI. THE
C     ORIGINAL AXIS IS NOW ROTATED BACK TAKING THE SCATTERING
C     VECTOR WITH IT. NOW WE HAVE THE SCATTERED
C     DIRECTION VECTOR (SX,SY,SZ).
C     WE USE EULER ANGLES TO PERFORM THE TRANSFORMATION.
      COMMON IX
C *** NORMALIZE THE DIRECTION TO A UNIT VECTOR (IN CASE IT WASN'T)
      S=SQRT(EX**2+EY**2+EZ**2)
      EX=EX/S
      EY=EY/S
      EZ=EZ/S
      BET=ACOS(EZ)
C *** THESE APPROXIMATIONS ARE ONLY NEEDED FOR COMPTON SCATTERING 
C *** FOR GAMMAS (BUT THEY WILL NOT HURT HERE)
      IF(ABS(BET).LT.0.027)ALF=0.0
      IF(ABS(BET).LT.0.027)GO TO 44
      ARG=EY/SIN(BET)
      AARG=ABS(ARG)
      IF(AARG.LT.1.0)GOTO 344
      ARG=ARG/(1.0001*AARG)
344   ALF=ASIN(ARG)
 44   CONTINUE
      SCO1=COS(ALF)*SIN(BET)+EX
      SCO1=ABS(SCO1)
      SCO2=ABS(EX)
      IF(SCO1.LT.SCO2)BET=-BET
      IF(SCO1.LT.SCO2)ALF=-ALF
      GAM=0.0
C     WE NOW HAVE THE EULER ANGLES OF ROTATION BETWEEN
C     Z-AXIS TO DIRECTION OF INITIAL PARTICLE.
      THET = ANGL
      CALL RANDM(RN1)
      FI = 6.2831853 * RN1
C     WE NOW HAVE SCATTERED THE PARTICLE FROM THE
C     Z-AXIS AND MUST ROTATE IT TO THE ORIGINAL UNSCATTERED
C     PARTICLE DIRECTION. CACULATE THE ROTATION MATRIX.
      R11 = COS(ALF)*COS(BET)*COS(GAM)-SIN(ALF)*SIN(GAM)
      R12 = COS(BET)*SIN(ALF)*COS(GAM)+COS(ALF)*SIN(GAM)
      R13 =-SIN(BET)*COS(GAM)
      R21 =-SI